## A simple model for demand and supply of geolocalized services in a city

### Load libraries and classes

In [1]:
import numpy as np
import pandas as pd

import os
import sys
nb_dir = os.path.dirname(os.getcwd()) ## TODO: find way to put this into some global settings
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from references import common_cfg, istat_kpi, city_settings

In [2]:
from src.models.city_items import AgeGroup, ServiceArea, ServiceType, SummaryNorm # enum classes for the model
from src.models.core import ServiceUnit, ServiceValues, ServiceEvaluator, \
    MappedPositionsFrame, DemandFrame, KPICalculator
from src.models.factories import UnitFactory
from src.models.process_tools import GridMaker, ValuesPlotter, JSONWriter
from src.models.city_items import get_random_pos

In [3]:
selectedCity = 'Torino'

### Load service data: locations, scales, other info for city

In [4]:
loaders=UnitFactory.make_loaders_for_city(city_settings.get_city_config(selectedCity))
loaders

{'Biblioteche': <src.models.factories.LibraryFactory at 0x7fa3dc3a0128>,
 'Farmacie': <src.models.factories.PharmacyFactory at 0x7fa3dc3a0550>,
 'Fermate TPL': <src.models.factories.TransportStopFactory at 0x7fa39f6bdc50>,
 'Scuole': <src.models.factories.SchoolFactory at 0x7fa3dc3a0438>}

In [5]:
# Initialise with a default lengthscale of 0.5 km
schoolUnits = loaders['Scuole'].load(meanRadius=0.5)

# Initialise with a default lengthscale of 0.3 km
pharmacyUnits = loaders['Farmacie'].load(meanRadius=0.3)

# Initialise with a default lengthscale of 0.3 km
## awful hack to quickly clean tpl file #FIXME
loaders['Fermate TPL']._rawData = loaders['Fermate TPL']._rawData[
    loaders['Fermate TPL']._rawData['route_type'].isin([0,1,3])].reset_index()

tplUnits = loaders['Fermate TPL'].load(meanRadius=0.3)

# Initialise with a default lengthscale of 0.5 km
#libraryUnits = bibliotecheLoader.load(meanRadius=0.8)

Location data found
Location data found
Location data found


### Demand import from Censimento Popolazione e Abitazioni (CPA) 2011

In [6]:
demandData = DemandFrame.create_from_istat_cpa(selectedCity)

bDemo = True
if bDemo:
    # demo mode, use a portion of the data
    demandTest = DemandFrame(demandData.sample(50).copy(), False)
    testUnits = schoolUnits[::10] + pharmacyUnits[::10] + tplUnits[::30] 
    calculator = KPICalculator(demandTest, testUnits, selectedCity)
else:
    calculator = KPICalculator(demandData, schoolUnits + pharmacyUnits + tplUnits, selectedCity)

### Evaluate the services offer at the demand points and average over neighbourhood

In [7]:
# compute and plot demand/supply interaction for localized services 
calculator.evaluate_services_at_demand() # this might take a while.
calculator.compute_kpi_for_localized_services()
pass

### Append Istat KPI and export averaged values by Neighbourhood-Service-AgeGroup to JSON

In [9]:
# compute istat kpi as well
calculator.compute_kpi_for_istat_values()


calculator.demand
# write KPI to json output
jsonMaker = JSONWriter(calculator)
jsonMaker.write_all_files_to_default_path()

Had to skip employment density
   IDquartiere                                      TransportStop
0            1  [nan, nan, 1.1126, 1.0289, 1.1905, 1.1267, 1.1...
1            2  [nan, nan, 0.413, 0.387, 0.4585, 0.4374, 0.460...
2            3  [nan, nan, 0.7708, 0.7398, 0.7027, 0.7294, 0.7...
3            4  [nan, nan, 0.6682, 0.7051, 0.6691, 0.6805, 0.6...
4            5  [nan, nan, 0.9517, 0.9429, 0.9042, 0.9917, 0.8...
5            6  [nan, nan, 0.3403, 0.3139, 0.3594, 0.3205, 0.4...
6            7  [nan, nan, 0.5697, 0.6204, 0.6696, 0.6039, 0.6...
7            8  [nan, nan, 0.4552, 0.5391, 0.4385, 0.3867, 0.4...
   IDquartiere                                             School  \
0            1  [nan, nan, 0.0458, 0.0197, 0.3841, nan, nan, n...   
1            2  [nan, nan, 0.2326, 0.0007, 0.1634, nan, nan, n...   
2            3  [nan, nan, 0.5849, 0.7247, 0.008, nan, nan, na...   
3            4  [nan, nan, 0.1925, 0.105, 0.0135, nan, nan, na...   
4            5  [nan, nan, 0.2